# TP 06 : Fine-tuning GPT-2 pour générer des Pokémon

**Objectif** : Fine-tuner GPT-2 français pour qu'il génère des descriptions de Pokémon

**Durée d'entraînement** : ~20 minutes

> **Pendant l'entraînement**, ouvrez le notebook **TP-06-Exploration.ipynb** pour explorer le dataset et comprendre les techniques utilisées.

---

## 1. Installation et imports

In [ ]:
# Installation des dépendances (Colab)
!pip install -q transformers datasets accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback
from datasets import load_dataset
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if not torch.cuda.is_available():
    print("\n⚠️  GPU non disponible ! L'entraînement sera très lent.")
    print("   Sur Colab : Runtime > Change runtime type > GPU")

---

## 2. Configuration

Ces paramètres sont pré-optimisés. Vous pouvez les modifier pour expérimenter.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║                       CONFIGURATION                              ║
# ╚══════════════════════════════════════════════════════════════════╝

MODEL_SIZE = "base"           # "small" (rapide) ou "base" (meilleur)
NUM_EPOCHS = 10               # Nombre d'epochs
MAX_LENGTH = 256              # Longueur max (tokens)
LEARNING_RATE = 5e-5          # Taux d'apprentissage

# Techniques d'optimisation (recommandé : laisser à True)
ADD_POKEMON_TOKENS = True     # Ajouter les noms au vocabulaire
FREEZE_LOWER_LAYERS = True    # Figer 50% des couches basses

print("Configuration chargée !")

---

## 3. Chargement des données et du modèle

In [ ]:
# Charger les noms de Pokémon
print("Chargement des noms de Pokémon...")
pokemon_names_ds = load_dataset("chris-lmd/pokemon-names-fr")
POKEMON_NAMES = [item["name"] for item in pokemon_names_ds["train"]]
print(f"  {len(POKEMON_NAMES)} noms chargés")

# Charger le dataset Pokepedia
print("\nChargement du dataset Pokepedia...")
dataset = load_dataset("chris-lmd/pokepedia-fr")
print(f"  {len(dataset['train']):,} articles au total")

# Filtrer pour ne garder que les vrais articles Pokémon
pokemon_names_set = set(name.lower() for name in POKEMON_NAMES)
train_dataset = dataset['train'].filter(
    lambda x: x.get('title', '').lower() in pokemon_names_set
)
print(f"  {len(train_dataset):,} articles Pokémon retenus")

In [ ]:
# Charger le modèle GPT-2 français
model_name = f"asi/gpt-fr-cased-{MODEL_SIZE}"
print(f"Chargement de {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

print(f"\n✅ Modèle chargé !")
print(f"   Paramètres : {sum(p.numel() for p in model.parameters()):,}")
print(f"   Couches : {model.config.n_layer}")

---

## 4. Techniques d'optimisation

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TECHNIQUE 1 : Ajouter les tokens Pokémon au vocabulaire
# ═══════════════════════════════════════════════════════════════

if ADD_POKEMON_TOKENS:
    print("Ajout des tokens Pokémon...")
    
    # Trouver le token de référence pour l'initialisation
    for ref in ["Pokémon", "Pokemon", "animal"]:
        tokens = tokenizer.encode(ref, add_special_tokens=False)
        if len(tokens) == 1:
            reference_id = tokens[0]
            print(f"  Token de référence : '{ref}'")
            break
    
    # Tokens à ajouter (ceux qui ne sont pas déjà uniques)
    new_tokens = [name for name in POKEMON_NAMES 
                  if len(tokenizer.encode(name, add_special_tokens=False)) > 1]
    
    # Sauvegarder l'embedding de référence
    with torch.no_grad():
        ref_embedding = model.transformer.wte.weight[reference_id].clone()
    
    # Ajouter et initialiser
    num_added = tokenizer.add_tokens(new_tokens)
    old_size = model.transformer.wte.weight.shape[0]
    model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
    
    with torch.no_grad():
        for i in range(num_added):
            noise = torch.randn_like(ref_embedding) * 0.01
            model.transformer.wte.weight[old_size + i] = ref_embedding + noise
    
    print(f"  ✅ {num_added} tokens ajoutés")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TECHNIQUE 2 : Figer les couches basses
# ═══════════════════════════════════════════════════════════════

if FREEZE_LOWER_LAYERS:
    print("Freezing des couches basses...")
    
    num_to_freeze = model.config.n_layer // 2
    
    # Figer embeddings
    for param in model.transformer.wte.parameters():
        param.requires_grad = False
    for param in model.transformer.wpe.parameters():
        param.requires_grad = False
    
    # Figer les N premières couches
    for i in range(num_to_freeze):
        for param in model.transformer.h[i].parameters():
            param.requires_grad = False
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"  Couches figées : {num_to_freeze}/{model.config.n_layer}")
    print(f"  ✅ Paramètres entraînables : {trainable:,} ({100*trainable/total:.0f}%)")

In [ ]:
# Déplacer sur GPU
model = model.to(device)
print(f"Modèle sur : {device}")

---

## 5. Préparation des données

In [ ]:
# Tokenization
def tokenize_function(examples):
    return tokenizer(
        examples['content'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

print("Tokenization...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

# Ajouter les labels (pour le language modeling)
tokenized_dataset = tokenized_dataset.map(
    lambda x: {'labels': x['input_ids'].copy()},
    batched=True
)

print(f"✅ {len(tokenized_dataset)} exemples prêts")

---

## 6. Entraînement

**⏱️ Durée estimée : ~20 minutes**

> Pendant ce temps, ouvrez **TP-06-Exploration.ipynb** pour explorer le dataset et les techniques !

In [ ]:
# Callback pour tracker la loss
class LossCallback(TrainerCallback):
    def __init__(self):
        self.losses = []
        self.steps = []
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            self.losses.append(logs["loss"])
            self.steps.append(state.global_step)

loss_callback = LossCallback()

# Configuration du trainer
batch_size = 2 if MODEL_SIZE == "base" else 4
grad_accum = 8 if MODEL_SIZE == "base" else 4

training_args = TrainingArguments(
    output_dir="./gpt2-pokemon",
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    callbacks=[loss_callback],
)

print("✅ Prêt pour l'entraînement !")

In [ ]:
# Résumé
print("╔" + "═" * 50 + "╗")
print("║" + " RÉSUMÉ ".center(50) + "║")
print("╠" + "═" * 50 + "╣")
print(f"║  Modèle : {model_name:<38} ║")
print(f"║  Dataset : {len(train_dataset):,} articles Pokémon{'':<20} ║")
print(f"║  Epochs : {NUM_EPOCHS}{'':<38} ║")
print(f"║  Tokens ajoutés : {ADD_POKEMON_TOKENS}{'':<30} ║")
print(f"║  Couches figées : {FREEZE_LOWER_LAYERS}{'':<30} ║")
print("╚" + "═" * 50 + "╝")

In [ ]:
# 🚀 LANCER L'ENTRAÎNEMENT
print("🚀 Fine-tuning en cours...")
print("\n⏱️  Allez explorer TP-06-Exploration.ipynb pendant ce temps !\n")

trainer.train()

print("\n✅ Fine-tuning terminé !")

In [ ]:
# 📊 Courbe de la loss
plt.figure(figsize=(10, 4))
plt.plot(loss_callback.steps, loss_callback.losses, 'b-', alpha=0.7)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Loss initiale : {loss_callback.losses[0]:.3f}")
print(f"Loss finale   : {loss_callback.losses[-1]:.3f}")
print(f"Réduction     : {(1 - loss_callback.losses[-1]/loss_callback.losses[0])*100:.0f}%")

In [ ]:
# Sauvegarder le modèle
trainer.save_model("./gpt2-pokemon-final")
tokenizer.save_pretrained("./gpt2-pokemon-final")
print("✅ Modèle sauvegardé dans ./gpt2-pokemon-final")

---

## 7. Test de génération

In [ ]:
def generate(prompt, temperature=0.7, max_length=150):
    """Génère du texte à partir d'un prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_k=50,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2,
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Test : Pikachu
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate(prompt, temperature=0.5))

In [ ]:
# Test : Dracaufeu
prompt = "Dracaufeu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate(prompt, temperature=0.5))

In [ ]:
# Test : Pokémon inventé
prompt = "Aqualis est un Pokémon de type Eau. Il"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate(prompt, temperature=0.7))

In [ ]:
# 🎮 Testez vos propres prompts !
mon_prompt = "Flamador est un Pokémon légendaire de type Feu et Dragon. Ce"

print(f"Prompt: {mon_prompt}")
print(f"\nGénération:")
print(generate(mon_prompt, temperature=0.7))

---

## 8. Pour aller plus loin

**Expérimentations suggérées :**
- Changer `temperature` : 0.3 (conservateur) → 1.0 (créatif)
- Essayer différents prompts
- Comparer avec `MODEL_SIZE = "small"`

**Voir aussi :**
- `TP-06-Exploration.ipynb` : Comprendre les techniques en détail